# Lin (2013)で学ぶSequential Variational Approximation でノンパラベイズ

Variational inference でDirichlet process infinite mixture model をしたくなったので適当に論文を見つけて読んで実装する.



## 1. Introduction

## 2. Related Work

## 3. Nonparametric Mixture Models

$$
D = \sum_{k=1}^{\infty}\pi_k\delta_{\phi_k}, \;
\operatorname{with} \pi_k = v_k\prod_{l=1}^{k-1}v_l, \;
v_k \propto \operatorname{Beta} \left( 1, \alpha_k\right), \;
\forall k = 1, 2, ... \tag{1} \\
$$

$$
D \propto DP\left( \alpha \mu\right), \;
\theta_i \propto \mu , \;
x_i \propto F\left( \cdot \mid \theta_i \right), \;
\forall i = 1, 2, ..., n \tag{2} \\
$$

$$
\begin{eqnarray}
z_{1:n} &\propto& \operatorname{CRP} \left( \alpha \right), \; \phi_k \propto \mu, \; \forall k = 1, 2, ... \\
x_{1:n} &\propto& F\left(\cdot \mid \phi_{z_i} \right), \; \forall i = 1, 2, ..., n  \tag{3}
\end{eqnarray}
$$

$$
p_{\operatorname {CRP}}\left( z_{1:n} \mid \alpha\right) =
\frac {\Gamma(\alpha) \alpha^K} {\Gamma(\alpha + n)} \prod_{k=1}^K \Gamma\left( | C_k| \right) \tag{4}
$$

## 4. Variational Approximation of Posterior

予測分布は以下のようになる.

$$
p\left( \theta'  \mid x_{1:n}\right) = \mathbb E_{D \mid x_{1:n}} \left\lbrack p\left( \theta' \mid D \right) \right\rbrack \tag {5}
$$

### 4.1. Posterior Analysis

$ D $ の事後分布.

$$
\begin{eqnarray}
D \mid \theta_{1:n} &\sim& \operatorname {DP} \left( \tilde {\alpha} \tilde {\mu}\right) \\
\tilde{\alpha} &=& \alpha + n \\
\tilde{\mu} &=& \frac {\alpha} {\alpha+n} \mu + \sum_{k=1}^K \frac {\left| C_k \right|} {\alpha + n} \delta_{\phi_k} \tag{6}
\end{eqnarray}
$$

$ p\left( D \mid z_{1:n}, x_{1:n} \right) $ からのサンプリングは以下のサンプリングと同じ.

$$
\begin{eqnarray}
\beta_0 D' + \sum_{k=1}^K \beta_k \delta_{\phi_k} , &&\\
&& with  \; D' \sim \operatorname {DP} \left( \alpha \mu  \right), \;
\left( \beta_0, \beta_1, ..., \beta_k \right) \sim \operatorname{Dir} \left( \alpha, m_1, ..., m_K \right), \;
\phi_k \sim \mu \mid _{C_k} \tag{7}
\end{eqnarray}
$$

ここで, $m_k = |C_k|$, $\mu \mid _{C_k}$ は 事後分布 $ \mu \mid _{C_k}\left( \operatorname d \theta \right) \propto \mu\left( \operatorname d \theta  \right) \prod_{i\in C_k} F \left( x_i \mid \theta \right)$

つまり$1:n$ までのデータが収まった中華料理店にもう１件データが追加された, という状態. $ \beta_k $ で既存のテーブルkに座り, $\beta_0$で新規テーブルが作られそこに座る. $\beta$の総和は1.

データ $ x_{1:n} $ が得られたときの混合分布 $ D $ の事後確率.

$$
p\left( D \mid x_{1:n} \right) = \sum_{z_{1:n}} p\left(z_{1:n} \mid x_{1:n} \right)p\left( D \mid x_{1:n}, z_{1:n}\right) \tag{8}
$$



$$
p \left( z_{1:n} \mid x_{1:n} \right)
\propto
    p_{\operatorname {CRP}} \left( z_{1:n} \mid \alpha \right)
    \prod_{k=1}^{K^{(z)}} \int \mu \left( \operatorname {d} \phi_{k} \right)
    \prod_{i \in C_k^{(z)}} F\left( x_i \mid \phi_k \right) \tag{9}
$$

## 4.2 Variational Approximation

(8)について, $p\left(z_{1:n} \mid x_{1:n} \right)$を因子分解可能とし, $ p\left( D \mid z_{1:n}, x_{1:n} \right) $ を(11)の確率過程で近似する.

$$
q\left(D \mid \rho, \nu \right) = \sum_{z_{1:n}} \prod_{i=1}^n \rho_i\left(z_i \right)q_{\nu}^{(z)}\left( D \mid z_{1:n}\right) \tag{10}
$$

$$
\begin{eqnarray}
q_{\nu}^{(z)} \left( D \mid z_{1:n}\right) \overset{d} {\sim} && \beta_0 D' + \sum_{k=1}^K\beta_k\delta{\phi_k} \\
&&with \; D' \sim \operatorname {DP} \left( \alpha \mu \right), \;
\left( \beta_0, \beta_1, ..., \beta_k \right) \sim \operatorname{Dir} \left( \alpha, m_1^{(z)}, ..., m_K^{(z)} \right), \;
\phi_k \sim \nu_k\tag{11}
\end{eqnarray}
$$

(10), (11)と(8), (9)はよく似ている. 違いは, 

1.  $p\left(z_{1:n} \mid x_{1:n} \right)$を因子分解可能とし, $ \prod_{i=1}^n \rho_i\left(z_i \right)$ で近似したこと
2.  $\mu |_{C_k}$ を $\nu_k$ で近似したこと 

$$
\mathbb E_{q\left( D \mid \rho, \nu  \right)}\left\lbrack p\left(\theta'  \mid D \right) \right\rbrack
=
    \frac {\alpha} {\alpha + n} \mu\left( \theta ' \right)
    + \sum_{k=1}^K \frac {\sum_{i=1}^n \rho_i\left( k \right)} {\alpha + n} \nu_k\left( \theta'\right) \tag{12}
$$

### 4.3 Sequential Approximation

$$
\tag{15}
\rho_{i+1} \propto \left\{ \begin{array}{ll}
    w_k^{(i)} \int_\theta F\left( x_{i+1} \mid \theta \right) \nu_k^{(i)} \left( d \theta \right) & (k \le K), \\
    \alpha \int_\theta F\left( x_{i+1} \mid \theta \right) \mu \left( d \theta \right) & (k = K+1), \\
  \end{array} \right.
$$

with $ w_i^{(i)} = \sum_{j=1}^i \rho_j\left(k \right)$, and 

$$
\tag{16}
\nu_{k}^{(i+1)}\left( d\theta \right) \propto \left\{ \begin{array}{ll}
    \mu\left( d\theta \right) \prod_{j=1}^{i+1} F\left( x_j \mid \theta \right)^{\rho_j(k)}
    & (k \le K), \\
    \mu\left( d\theta \right) F\left( x_{i+1} \mid \theta \right)^{\;\rho_{i+1}(k)}
    & (k = K+1), \\
  \end{array} \right.
$$

$\rho_1 = 1, K = 1$ から開始する, $i$を増やすたびに, $\rho_{i+1}$の計算, $w_k$ の更新, Kの更新, $\nu$の更新(, と$\rho$の更新も?)を繰り返す. Kをインクリメントするかどうかは, $\rho_{i+1}\left( K+1 \right) $ がしきい値以上であるか否かで判定する. Kをインクリメントしない場合は, $\rho_{i+1}\left( K+1 \right) $は不要なので削除, $\rho_{i+1}\left( 1,..., K \right) $の総和が1になるように標準化する.

## Normal-Wishart分布を基底分布にしてinfinite mixture multivariate normal modelを実装してみる.

多変量正規分布のInfinite mixture モデルを考える. 分布のパラメータとして, 平均と精度行列を推定する.

$$
\begin{eqnarray}
\nu_k &=& \left(\boldsymbol \mu_k, \boldsymbol \Lambda_k \right) \sim
    \mathcal N \left(\boldsymbol \mu_{k} \mid \mathbf m_{0}, \left(\beta_{0} \boldsymbol \Lambda_{k} \right)^{-1}\right)
    \mathcal W\left(\boldsymbol \Lambda_{k} \mid \mathbf W_{0}, \nu_{0}\right)
\end{eqnarray}
$$

(15)を書き下す. まず$k \le K$の場合を考える.

$$
\begin{eqnarray}
w_k^{(i)} \int_\theta F\left( x_{i+1} \mid \theta \right) \nu_k^{(i)} \left( d \theta \right)
&=&
    w_k^{(i)} \int
    \mathcal N \left( x_{i+1} \mid \boldsymbol \mu_k, \boldsymbol \Lambda_k^{-1} \right)
    \mathcal N \left(\boldsymbol \mu_{k} \mid \mathbf m_{0}, \left(\beta_{0} \boldsymbol \Lambda_{k} \right)^{-1}\right)
    \mathcal W\left(\boldsymbol \Lambda_{k} \mid \mathbf W_{0}, \nu_{0}\right)
    \operatorname d \boldsymbol \mu \operatorname d \boldsymbol \Lambda 
\end{eqnarray}
$$

(16)の上の式を考える. (16)の下の式は上の式の変形なので, 上の式さえ解ければ良い.

$$
\begin{eqnarray}
\mu\left( d\theta \right) \prod_{j=1}^{i+1} F\left( x_j \mid \theta \right)^{\rho_j(k)}
&=&
    \prod_{j=1}^{i+1}\mathcal N \left( x_{j} \mid \boldsymbol \mu_k, \boldsymbol \Lambda_k^{-1} \right)^{\;\rho_j(k)}
    \mathcal N \left(\boldsymbol \mu_{k} \mid \mathbf m_{0}, \left(\beta_{0} \boldsymbol \Lambda_{k} \right)^{-1}\right)
    \mathcal W\left(\boldsymbol \Lambda_{k} \mid \mathbf W_{0}, \nu_{0}\right)
\end{eqnarray}
$$

多変量正規分布+事前分布(Normal-Wishart分布)の形だが, データの尤度の肩に$\rho$が乗っているのが[以前解いたもの](http://kazufusa1484.hatenablog.com/entry/2018/07/27/121549)と異なっている. 

$ \sum_{j=1}^{i+1}\rho_j\left(k\right) = r_{i+1, k}と重み付け平均\overline {\mathbf x} = \sum_{j=1}^{i+1} \rho_j(k) \textbf x_j  / r_{i+1, k} $ を導入する.　以前のものの$N$を$r_{i+1, k}$に置き換えるだけで解決する.